
# SixtyFour AI — Enrich Company

This notebook is your guide to using the Sixtyfour API `enrich-company`.

The `enrich-company` endpoint allows you to perform deep research on a company. It recursively reads through relevant sources (websites, profiles, public signals, socials) and can answer questions like: **Who owns this company?** **What is their approximate team size?** **What social accounts do they operate?** **What is their tech stack?** **Which decision-makers should I contact?**

**Tip:** You can change the company in the parameters at the top. For best results, include at least the **company name** and **website** or a strong identifier (e.g., domain).



## Prerequisites & Secrets

### Get your SixtyFour API key
Follow these steps to create and copy your API key:

**1) Sign in**  
Go to https://app.sixtyfour.ai/login and log in or sign up.

**2)** Enter an organization name once you've signed up and click **Create Organization** and get your $25 in credits.

**3)** You’ll land on the dashboard/usage page with credits. In the left sidebar, open **Keys** → click **Create new key** → name it → **Create key** → copy the key value.

### If you’re in Google Colab
- Click the 🔑 (Secrets) icon in the left sidebar.  
- Add a secret named `SIXTYFOUR_API_KEY` and paste your key.  
- Make sure the notebook can access the secret.


In [3]:

# --- Parameters (you can change these) ---
COMPANY_NAME = "Sixtyfour AI"
COMPANY_WEBSITE = "sixtyfour.ai"
# COMPANY_ADDRESS = "fill this in for better context"
# COMPANY_PHONE = "fill this in for better context"

# Optional controls
FIND_PEOPLE = True

import os, json, requests

API_KEY = os.getenv("SIXTYFOUR_API_KEY", "")  # fallback if already set locally
try:
    from google.colab import userdata
    secret_val = userdata.get("SIXTYFOUR_API_KEY")  # raises if not granted/not created
    if secret_val:
        os.environ["SIXTYFOUR_API_KEY"] = secret_val
        API_KEY = secret_val
except Exception:
    pass  # Not in Colab or secret not available; rely on existing env var

BASE_URL = "https://api.sixtyfour.ai"
SYNC_ENDPOINT = f"{BASE_URL}/enrich-company"
TIMEOUT_S = 600

print("✅ Setup complete (key detected, endpoint ready).")


✅ Setup complete (key detected, endpoint ready).



## Enrich_Company Request

For most use cases, `enrich-company` is the perfect tool. You send a request and get enriched company data back with a single call.

The request body requires two main components:

- **target_company**: The initial data you have on the company. The more context you provide, the more focused and correct the research will be (e.g., name + website + address + phone).
- **struct**: The fields you want extracted. The key is the field name you want returned; the value is either a description string or an object with description and an optional type to guide the agent.

Responses may take between 5-10 minutes depending on how deep the research goes.


In [4]:

target_company = {
    "company_name": COMPANY_NAME,
    "website": COMPANY_WEBSITE,
    # "address": COMPANY_ADDRESS,
    # "phone_number": COMPANY_PHONE,
}

# You can optionally specify types for predictable outputs (e.g., {"type": "int"}); otherwise simple descriptions are fine.
# Below mixes untyped (plain descriptions) and typed (with {"type": ...}) fields.
struct = {
    "instagram_url": "Instagram url for the photography company",
    "is_remote_friendly": "Is the company remote-friendly?",
    "hq_location": "HQ location (city, region, country)",
    "linkedin_url": "LinkedIn company page",

    # Typed structures
    "num_employees": {"description": "Approximate number of employees", "type": "int"},
    "founded_year": {"description": "Year the company was founded", "type": "int"},
    "tech_stack": {"description": "Core technologies used", "type": "list[string]"},
    "revenue_estimate": {"description": "Estimated annual revenue", "type": "string"}
    # You can also add your own fields here to find more information about the company.

}

payload = {
    "target_company": target_company,
    "struct": struct,
    "find_people": FIND_PEOPLE,
}

resp = requests.post(
    SYNC_ENDPOINT,
    headers={"x-api-key": API_KEY, "Content-Type": "application/json"},
    json=payload,
    timeout=TIMEOUT_S,
)

if resp.ok:
    data = resp.json()
    print("✅ Success! Top-level keys:", list(data.keys()))
    print("\nStructured data:")
    print(json.dumps(data.get("structured_data", {}), indent=2))
else:
    print("❌ Request failed:", resp.status_code, resp.text[:800])


✅ Success! Top-level keys: ['notes', 'structured_data', 'findings', 'references', 'confidence_score']

Structured data:
{
  "instagram_url": "",
  "is_remote_friendly": "",
  "hq_location": "San Francisco, California, United States",
  "linkedin_url": "https://www.linkedin.com/company/sixtyfourai",
  "num_employees": 3,
  "founded_year": 2025,
  "tech_stack": "Python, FastAPI, Docker, Machine Learning APIs, AI-driven API services",
  "revenue_estimate": "",
  "company_name": "Sixtyfour AI",
  "website": "https://sixtyfour.ai",
  "leads": [
    {
      "name": "Saarth Shah",
      "email": "saarth@sixtyfour.ai",
      "company": "Sixtyfour AI",
      "title": "CEO & Co-Founder",
      "phone": "",
      "score": "9.5",
      "linkedin": "https://www.linkedin.com/in/saarthshah"
    },
    {
      "name": "Christopher Price",
      "email": "",
      "company": "Sixtyfour AI",
      "title": "Founder / COO",
      "phone": "",
      "score": "9",
      "linkedin": "https://www.linkedin.co


### How to change values

- **Pick a different company:** edit `COMPANY_NAME` and `COMPANY_WEBSITE` (address/phone optional).  
- **Ask for different fields:** edit `struct`. Mix untyped (description strings) and typed objects with `description` and `type` if you need specific formats.  


## Example structs you can run (copy/paste)
Pick one of these, assign to `struct`, then call the same `requests.post` as above.


In [5]:

# Product/Engineering mapping
example_tech_struct = {
    "company_name": "Official company name",
    "domain": {"description": "Primary domain (e.g., example.com)", "type": "string"},
    "tech_stack": {"description": "Key technologies used", "type": "list[string]"},
    "engineering_team_size": {"description": "Approximate size of engineering org", "type": "int"},
    "github_org": {"description": "Company GitHub org URL", "type": "string"},
    "notable_repos": {"description": "Top repositories (names/links)", "type": "list[string]"}
}

# Sales prospecting (company-level)
example_sales_struct = {
    "company_name": "Official company name",
    "website": "Website URL",
    "num_employees": {"description": "Approximate employee count", "type": "int"},
    "hq_location": {"description": "Headquarters (city, region, country)", "type": "string"},
    "industry": {"description": "Primary industry/sector", "type": "string"},
    "recent_news": {"description": "Recent press or announcements", "type": "list[string]"},
    "key_buyers": {"description": "Buyer functions to target (e.g., Ops, RevOps)", "type": "list[string]"}
}

# Marketing
example_marketing_struct = {
    "brand_handles": {"description": "Social profiles (Twitter, Instagram, LinkedIn)", "type": "list[string]"},
    "content_topics": {"description": "Topics they post about", "type": "list[string]"},
    "audience_size": {"description": "Aggregate audience size estimate", "type": "int"},
    "newsletter_url": {"description": "Newsletter or blog URL", "type": "string"},
    "paid_channels": {"description": "Paid marketing channels used", "type": "list[string]"}
}

# Partnerships / BD
example_bd_struct = {
    "strategic_initiatives": {"description": "Key initiatives they own", "type": "list[string]"},
    "notable_customers": {"description": "Notable customers/logos", "type": "list[string]"},
    "ecosystem_partners": {"description": "Ecosystem/integration partners", "type": "list[string]"},
    "regions": {"description": "Regions the company operates in", "type": "list[string]"}
}

# Investor / diligence
example_investor_struct = {
    "founded_year": {"description": "Year founded", "type": "int"},
    "founders": {"description": "Founders' names", "type": "list[string]"},
    "funding_rounds": {"description": "Funding rounds and amounts", "type": "list[string]"},
    "lead_investors": {"description": "Lead investors by round", "type": "list[string]"},
    "revenue_estimate": {"description": "Estimated revenue (latest FY)", "type": "string"},
    "num_employees": {"description": "Approximate employee count", "type": "int"}
}



## Troubleshooting & Next steps

**Remember:**  
- You can request up to 100 fields in one `struct`  
- Adding more fields doesn't increase API usage but adding rows does.
- Rate limit: **500 requests per minute** per API key  
- Don't be vague — “Sixtyfour” vs “Sixtyfour AI (YC X25) from SF”  
- Provide context — More details = better results  


**Next steps:** try a different `struct`, point the CSV output to your CRM, or move this into an automated script.
